# Song Recommendation Using Spotify

### Accessing the Spotify API

In [32]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd


cid = "c3dddfed85d44508b9c82203d292cfea" 
secret = "4ed920860f33464aa2e97182c7d0d18e"

client_credentials_manager = SpotifyClientCredentials(client_id = cid, client_secret = secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### Inputting the Song Title

In [35]:
track = input ("Track: ")

Track: talk khalid


### Collecting Track Data 

In [36]:
track_object = sp.search(track, limit=1, type='track')
track_id = track_object['tracks']['items'][0]['id']
track_name = track_object['tracks']['items'][0]['name']
artist_name = track_object['tracks']['items'][0]['album']['artists'][0]['name']
track_info = sp.audio_features(track_id)[0]

danceability = track_info['danceability']
energy = track_info['energy']
loudness = track_info['loudness']
speechiness = track_info['speechiness']
acousticness = track_info['acousticness']
instrumentalness = track_info['instrumentalness']
liveness = track_info['liveness']
valence = track_info['valence']
tempo = track_info['tempo']

print('Your track info:')
track_id_dict = {track_id: [(track_name, artist_name), 0, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo]}
track_data = pd.DataFrame.from_dict(track_id_dict, orient = 'index')
track_data.columns = ['name&artist', 'count', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
track_data

Your track info:


,name&artist,count,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0rTV5WefWd1J3OwIheTzxM,"(Talk, Khalid)",0,0.9,0.4,-8.443,0.124,0.0501,0,0.0876,0.338,136.001


### Collecting 2,000 song data from Spotify using their recommendations() function

In [37]:
import time

songs_over_time = []
track_id = track_object['tracks']['items'][0]['id']

for attempt in range(100):
    recommended = sp.recommendations(seed_tracks=[track_id], limit=20)
    songs_over_time.append(recommended['tracks'])
    time.sleep(1)

print("Done collecting data.")


Done collecting data.


### Arranging Data 

In [38]:
id_dict = {}
for i in range(100):
    for song_dict in songs_over_time[i]:
        song_id = song_dict['id']

        if song_id in id_dict:
            id_dict[song_id][1] += 1
        else:
            song_name = song_dict['name']
            artist_name = song_dict['album']['artists'][0]['name']
            track_info = sp.audio_features(song_id)[0]
            danceability = track_info['danceability']
            energy = track_info['energy']
            loudness = track_info['loudness']
            speechiness = track_info['speechiness']
            acousticness = track_info['acousticness']
            instrumentalness = track_info['instrumentalness']
            liveness = track_info['liveness']
            valence = track_info['valence']
            tempo = track_info['tempo']
            
            id_dict[song_id] = [(song_name, artist_name), 1, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo]

print('Done arranging data.')
# id_data = pd.DataFrame.from_dict(id_dict, orient='index')
# id_data.columns = ['name&artist', 'count', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
# id_data = id_data.sort_values(by= ['count'])
# id_data.tail()

Done arranging data.


###  Normalizing Data

In [22]:
from sklearn import preprocessing as pp

normalized_dict = {}
id_list = []
danceability_list = []
energy_list = [] 
loudness_list = [] 
speechiness_list = [] 
acousticness_list = [] 
instrumentalness_list = []
liveness_list = []
valence_list = []
tempo_list = []

for key, value in id_dict.items():
    id_list.append(key)
    danceability_list.append(value[2])
    energy_list.append(value[3])
    loudness_list.append(value[4])
    speechiness_list.append(value[5])
    acousticness_list.append(value[6])
    instrumentalness_list.append(value[7])
    liveness_list.append(value[8])
    valence_list.append(value[9])
    tempo_list.append(value[10]) 

danceability_list = pp.scale(danceability_list)
energy_list = pp.scale(energy_list)
loudness_list = pp.scale(loudness_list)
speechiness_list = pp.scale(speechiness_list)
acousticness_list = pp.scale(acousticness_list)
instrumentalness_list = pp.scale(instrumentalness_list)
liveness_list = pp.scale(liveness_list)
valence_list = pp.scale(valence_list)
tempo_list = pp.scale(tempo_list)


for i in range(len(id_dict)):
    key = id_list[i]
    name_and_artist = id_dict[key][0]
    count = id_dict[key][1]
    danceability = danceability_list[i]
    energy = energy_list[i]
    loudness = loudness_list[i] 
    speechiness = speechiness_list[i]
    acousticness = acousticness_list[i] 
    instrumentalness = instrumentalness_list[i]
    liveness = liveness_list[i]
    valence = valence_list[i]
    tempo = tempo_list[i]
    
    normalized_dict[key] = [name_and_artist, count, danceability, energy, loudness, speechiness, acousticness, instrumentalness, liveness, valence, tempo]


print('Done normalizing data.')
# normalized_data = pd.DataFrame.from_dict(normalized_dict, orient='index')
# normalized_data.columns = ['name', 'count', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
# normalized_data = normalized_data.sort_values(by= ['count'])

# normalized_data.tail()

Done normalizing data.


### Assigning Index to Tracks

In [39]:
def positive(n):
    if n < 0:
        return n * -1
    return n


diff_dict = {}

for key, value in id_dict.items():
    for track_key, track_value in track_id_dict.items():
        track_id = track_key
        track_value = track_value
        track_name = track_value[0]

    diff = 0
    for i in range(2, 11):
        to_add = positive(value[i] - track_value[i])
        diff += to_add
        
    name_and_artist = value[0]
    name = name_and_artist[0]
    artist = name_and_artist[1]
    count = value[1]
    index = diff / count
    
    diff_dict[name] = [artist, index]
    

print('Done assigning index.')
# diff_data = pd.DataFrame.from_dict(diff_dict, orient='index')
# diff_data.columns = ['artist', 'index']
# diff_data = diff_data.sort_values(by= ['index'])

# normalized_data.head()

Done assigning index.


### Final Recommendations !!

In [40]:
diff_data = pd.DataFrame.from_dict(diff_dict, orient='index')
diff_data.columns = ['artist', 'index']
diff_data = diff_data.sort_values(by= ['index']).nsmallest(21, 'index')
diff_data

,artist,index
Talk,Khalid,0.000000
STFU,mansionz,0.194792
Fish N Grits (feat. Travis Scott),Wale,0.195807
Nympho,Jeremih,0.252036
The Flute Song,Russ,0.324830
Poledancer (feat. Megan Thee Stallion),Wale,0.373750
773 Love,Jeremih,0.374757
ABCD (Friend Zone),PnB Rock,0.546340
"Iced Out My Arms (feat. Future, Migos, 21 Savage & T.I.)",DJ Khaled,0.549458
REEL IT IN,Aminé,0.549729
